In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm

d:\Envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda')

In [3]:
class EfficientNetV2LOCR(nn.Module):
    def __init__(self, len_chars, shape_inp_img):
        super(EfficientNetV2LOCR, self).__init__()
        self.len_chars = len_chars
        self.shape_inp_img = shape_inp_img

        self.base_model = timm.create_model('efficientnetv2_l',
                                            pretrained=False,
                                            num_classes=0,
                                            features_only=True).to(device)
        
        self.dropout = nn.Dropout(0.3)

        self.lstm = nn.LSTM(
            input_size=self._get_lstm_input_size(),
            hidden_size=256,
            num_layers=1,
            batch_first=True,
            bidirectional=True,
            dropout=0.3 if True else 0.0
        ).to(device)

        self.batch_norm = nn.BatchNorm1d(256).to(device)
        
        self.fc = nn.Linear(256, len_chars + 1).to(device)  

    def _get_lstm_input_size(self):
        dummy_input = torch.zeros(1, *self.shape_inp_img).to(device)
        with torch.no_grad():
            features = self.base_model(dummy_input)
            _, C, H, W = features[-1].shape
            return C  

    def forward(self, x):
        features = self.base_model(x)
        feature = features[-1] 

        batch_size, C, H, W = feature.size()
        x = feature.permute(0, 2, 3, 1).contiguous().view(batch_size, H * W, C)

        x = self.dropout(x)

        lstm_out, _ = self.lstm(x)
        lstm_out_forward = lstm_out[:, :, :256]
        lstm_out_backward = lstm_out[:, :, 256:]
        x = (lstm_out_forward + lstm_out_backward) / 2

        x = x.permute(0, 2, 1).contiguous()
        x = self.batch_norm(x)
        x = x.permute(0, 2, 1).contiguous()

        x = self.fc(x)
        x = F.log_softmax(x, dim=2)  
        x = x.permute(1, 0, 2)  

        return x


In [4]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import os

In [5]:
vocabulary = "-1234567890ABEKMHOPCTYX"

char_to_idx = {char: idx for idx, char in enumerate(vocabulary)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

char_to_idx['PAD'] = len(char_to_idx)
idx_to_char[len(char_to_idx) - 1] = 'PAD'

In [6]:
class PlateDataset(Dataset):
    def __init__(self, dir_path: str, transform=None):
        self.dir_path = dir_path
        self.transform = transform

        self.image_files = [f for f in os.listdir(dir_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.image_files.sort()

    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        """
        Возвращаем изображение и метку по индексу

        Args:
            idx (int): Индекс элемента.
        
        Returns:
            tuple: (изображение, метка в виде индексов)
        """

        img_name = self.image_files[idx]
        img_path = os.path.join(self.dir_path, img_name)

        image = Image.open(img_path).convert('RGB')
        image = np.array(image)

        if self.transform:
            augmented = self.transform(image)
            image = augmented['image']

        label = os.path.splitext(img_name)[0]

        indices = self.label_to_indices(label=label)

        return image, torch.tensor(indices, dtype=torch.long)

    def label_to_indices(self, label):
        indices = [char_to_idx[char] for char in label if char in char_to_idx and char != 'PAD']
        return indices


In [7]:
class PlatesRecognized:
    def __init__(self):
        self.false_rec_car_plates = 0
        self.all_car_plates = 0

    def update_state(self, y_true, y_pred):
        max_length = min(y_true.size(1), y_pred.size(1))  
        y_true, y_pred = y_true[:, :max_length], y_pred[:, :max_length]

        res_in_batch_bool = torch.logical_not(y_true.eq(y_pred))
        res_in_batch_num = res_in_batch_bool.sum(dim=1)
        self.false_rec_car_plates += (res_in_batch_num > 0).sum().item()
        self.all_car_plates += y_true.size(0)

    def result(self):
        return 1.0 - self.false_rec_car_plates / self.all_car_plates if self.all_car_plates else 0.0

    def reset(self):
        self.false_rec_car_plates = 0
        self.all_car_plates = 0

In [8]:
class SymbolsRecognized:
    def __init__(self):
        self.false_rec_symbols = 0
        self.all_symbols = 0

    def update_state(self, y_true, y_pred):
        max_length = min(y_true.size(1), y_pred.size(1))  
        y_true, y_pred = y_true[:, :max_length], y_pred[:, :max_length]

        res_in_batch_bool = torch.logical_not(y_true.eq(y_pred))
        res_in_batch_num = res_in_batch_bool.sum()
        self.false_rec_symbols += res_in_batch_num.item()
        self.all_symbols += y_true.numel()

    def result(self):
        return 1.0 - self.false_rec_symbols / self.all_symbols if self.all_symbols else 0.0

    def reset(self):
        self.false_rec_symbols = 0
        self.all_symbols = 0

In [9]:
transform = A.Compose([
    A.Resize(width=200, height=100),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=10, p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2(),
])


train_dir = r'D:\Thesis\cache\storage_manager\datasets\ds_2ff4fcefd16749ac9485120c54e61a37\train\img'

train_dataset = PlateDataset(dir_path=train_dir, transform=transform)

def collate_fn(batch):
    images, labels = zip(*batch)
    images = torch.stack(images, 0)

    max_label_length = max(len(label) for label in labels)
    labels_padded = torch.zeros(len(labels), max_label_length, dtype=torch.long)

    for i, label in enumerate(labels):
        labels_padded[i, :len(label)] = label

    label_lengths = torch.tensor([len(label) for label in labels], dtype=torch.long)
    return images, labels_padded, label_lengths



batch_size = 16

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    pin_memory=True
)



In [10]:
from clearml import Task

task = Task.init(project_name='thesis', task_name='OCR_EfficientNetV2L')

ClearML Task: overwriting (reusing) task id=d0b2f5278dfe404ca408ed926995c600
2024-10-25 23:26:20,337 - clearml.Repository Detection - WARNING - Could not read Jupyter Notebook: No module named 'nbconvert'
2024-10-25 23:26:20,340 - clearml.Repository Detection - WARNING - Please install nbconvert using "pip install nbconvert"
ClearML results page: http://127.0.0.1:8080/projects/60b6a1102bdc4de3bdc5a20e918b0379/experiments/d0b2f5278dfe404ca408ed926995c600/output/log
2024-10-25 23:26:20,380 - clearml.Task - INFO - Storing jupyter notebook directly as code


In [11]:
num_classes = len(char_to_idx) - 1

model = EfficientNetV2LOCR(len_chars = num_classes, shape_inp_img=(3, 50, 200)).to(device)


RuntimeError: No pretrained weights exist for efficientnetv2_m. Use `pretrained=False` for random init.

In [12]:
ctc_loss = nn.CTCLoss(blank=num_classes, zero_infinity=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
plate_metric = PlatesRecognized()
symbol_metric = SymbolsRecognized()

In [14]:
for epoch in range(10):
    plate_metric.reset()
    symbol_metric.reset()
    
    for batch_idx, (images, labels_flat, label_lengths) in enumerate(train_dataloader):
        images = images.to(device)
        labels_flat = labels_flat.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        batch_size = images.size(0)
        input_lengths = torch.full(size=(batch_size,), fill_value=outputs.size(0), dtype=torch.long).to(device)
        loss = ctc_loss(outputs, labels_flat, input_lengths, label_lengths)
        loss.backward()
        optimizer.step()

        y_pred_decoded = outputs.argmax(dim=2).permute(1, 0)
        plate_metric.update_state(labels_flat, y_pred_decoded)
        symbol_metric.update_state(labels_flat, y_pred_decoded)

        task.logger.report_scalar("Batch Loss", "Batch", loss.item(), batch_idx + epoch * len(train_dataloader))

    task.logger.report_scalar("Epoch Loss", "Epoch", loss.item(), epoch)
    task.logger.report_scalar("Plates Accuracy", "Epoch", plate_metric.result(), epoch)
    task.logger.report_scalar("Symbols Accuracy", "Epoch", symbol_metric.result(), epoch)

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}, Plates Accuracy: {plate_metric.result()}, Symbols Accuracy: {symbol_metric.result()}")

Epoch 1, Loss: 2.530985116958618, Plates Accuracy: 0.0, Symbols Accuracy: 0.04111184645929844


KeyboardInterrupt: 